In [23]:
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV    
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_20newsgroups
import xgboost as xgb
from sklearn.model_selection import cross_validate

In [2]:
# Carregando a base de dados
categories = ['alt.atheism', 'comp.graphics','rec.sport.hockey', 'sci.space']
ng_train = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'), categories=categories)
# ng_train.data lista de string, cada string é um documento
ng_train.data[0]

'\n\n\n\nIn fact, you can all direct your ire at the proper target by ingoring NASA \naltogether.  The rocket is a commercial launch vechicle - a Conestoga flying \na COMET payload.  NASA is simply the primary customer.  I believe SDIO has a\nsmall payload as well.  The advertising space was sold by the owners of the\nrocket, who can do whatever they darn well please with it.  In addition, these\nanonymous "observers" had no reason to be startled.  The deal made Space News\nat least twice. \n\n\n\n\nI think its only fair to find that out before everyone starts having a hissy\nfit.  The fact that they bothered to use the conditional tense suggests that\nit has not yet been approved.\n\n'

In [3]:
# rótulos, vetor de rótulos
y = ng_train.target
y

array([3, 2, 2, ..., 1, 2, 3])

In [19]:
# criando pipeline com préprocessamento
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model',xgb.XGBClassifier(
        objective="multi:softprob", random_state=42)
    )
])

In [14]:
%%time
pipe.fit(ng_train.data, y);

CPU times: user 45.1 s, sys: 35.7 ms, total: 45.1 s
Wall time: 6.04 s


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1,
                               objective='multi:softprob', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [15]:
s = ["God is good!",'computer image jpeg','sport hocker']
out = pipe.predict(s)
out

array([0, 1, 2])

In [16]:
_ = ['team leaders']
pipe.predict(_)

array([2])

In [17]:
y

array([3, 2, 2, ..., 1, 2, 3])

In [18]:
categories

['alt.atheism', 'comp.graphics', 'rec.sport.hockey', 'sci.space']

In [21]:
# X = TfidfVectorizer().fit_transform(ng_train.data)

In [27]:
%%time
res = cross_validate(pipe, ng_train.data, y)

CPU times: user 3min 9s, sys: 128 ms, total: 3min 9s
Wall time: 25.9 s


In [28]:
res

{'fit_time': array([4.84767294, 5.1755538 , 5.55711269, 4.94008565, 4.88781619]),
 'score_time': array([0.08110237, 0.12230229, 0.08398366, 0.09188986, 0.08669233]),
 'test_score': array([0.77212389, 0.76769912, 0.81596452, 0.78713969, 0.80709534])}